<a href="https://colab.research.google.com/github/dado-beo/esercizi_python/blob/main/Tipi_di_processi_paralleli_ns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## 1) Processi Totalmente Ignari:

In questo caso, i processi non sono consapevoli l'uno dell'altro
e operano indipendentemente senza alcuna comunicazione diretta o scambio di informazioni.
Esempio: Parallelismo di dati. Supponiamo di avere un'immagine divisa in pixel.
Ogni processo si occupa di elaborare i dati in un determinato set di pixel senza
avere alcuna consapevolezza degli altri processi che elaborano gli altri pixel.

In [ ]:
from multiprocessing import Process
"""
multiprocessing è una libreria per la creazione, la comunicazione e la sincronizzazione
tra processi nella programmazione parallela e concorrente
Process è una classe per creare processi eseguendo una funzione (o metodo) specificata come target.
"""

def process_function(data):
  result = data * 2
  print(result)

if __name__ == "__main__": # Verifica se il modulo è eseguito come script principale
  data_list = [1, 2, 3, 4]
  processes = []

  for data in data_list:
    p = Process(target = process_function, args = (data,)) # Crea un processo per eseguire process_function con data come argomento
    processes.append(p)
    p.start() # Avvia l'esecuzione del processo separato

  for p in processes:
    p.join() # Blocca il processo principale fino a quando il processo separato non termina

## 2) Processi Indirettamente a Conoscenza Uno dell'Altro:

In questo caso, i processi potrebbero non conoscere direttamente gli altri,
ma possono interagire attraverso uno scambio indiretto di informazioni,
ad esempio, attraverso una struttura dati condivisa o un intermediario.
Esempio: Una coda condivisa. I processi possono mettere dati in una coda condivisa
e prelevarli successivamente. Anche se i processi non conoscono direttamente gli altri processi,
condividono una struttura dati (la coda) per scambiare informazioni.

In [ ]:
"""
Queue: classe che rappresenta una coda condivisa tra i processi.
È utilizzata per consentire la comunicazione tra processi,
consentendo loro di scambiarsi dati in modo sicuro.
put(item): Aggiunge un elemento alla coda.
get(): Rimuove e restituisce un elemento alla coda.
empty(): Restituisce True se la coda è vuota, altrimenti restituisce False.
full(): restituisce True se la coda è piena, altrimenti restituisce False.
qsize(): Restituisce il numero di elementi presenti nella coda.
close(): Chiude la coda.
-- current_process: funzione che restituisce un oggetto Process
che rappresenta il processo in esecuzione.
"""

def process_id():
  print(f"Server PID: {os.getpid()}, Process Name: {current_process().name}, Process PID: {current_process().pid}")

def process_function(data, result_queue):
  process_id()
  print("Elabora: ",data)
  result = data * 2
  result_queue.put(result)

if __name__ == "__main__": # Verifica se il modulo è eseguito come script principale
  data_list = [1, 2, 3, 4]
  result_queue = Queue() # Coda per memorizzare i risultati
  processes = []

  for data in data_list:
    p = Process(target = process_function, args = (data, result_queue)) # Crea un processo per eseguire process_function
    processes.append(p)
    p.start()

  for p in processes:
    p.join()

  print("Il main stampa i risultati")
  while not result_queue.empty():
    result = result_queue.get()
    print(result)

## 3) Processi direttamente a conoscenza
In questo caso, i processi sono consapevoli l'uno dell'altro e possono comunicare direttamente.
Esempio: Comunicazione diretta. I processi comunicano utilizzando messaggi diretti
o altri meccanismi di comunicazione.
Ad esempio, uno scenario di scambio di dati tra server e client
o tra processi all'interno di un'applicazione che utilizza canali di comunicazione diretti.

In [ ]:
import os # il modulo fornisce funzionalità per interagire con il sistema operativo
from multiprocessing import Process, current_process, Pipe

def process_id():
  print(f"Server PID: {os.getpid()}, Process Name: {current_process().name}, Process PID: {current_process().pid}")

def process_function(conn):
  process_id()
  print("Elaboro il dato ricevuto ed invio il risultato")
  data_recived = conn.recv()
  result = data_recived * 2
  conn.send(result)
  conn.close()

if __name__ == "__main__":
  process_id()
  print("Creo una connessione e un processo figlio")
  parent_conn, child_conn = Pipe()
  data = 5
  p = Process(target = process_function, args=(child_conn,))
  p.start()
  parent_conn.send(data)
  result = parent_conn.recv()
  p.join()
  process_id()
  print("Stampo il risultato ricevuto")
  print(result)